In [ ]:
#!pip install tensorflow-gpu
# Downgrade 
#smart_open to 1.10.0 -> https://github.com/RaRe-Technologies/smart_open/issues/475
# python -m pip install -U smart_open==1.10.0

In [ ]:
import os
%load_ext tensorboard

In [ ]:
###########################################
# How to make deterministic experiments?
###########################################

# Main Sources:
    # 1) https://github.com/NVIDIA/tensorflow-determinism
    # 2) https://pypi.org/project/tensorflow-determinism/#description
            # There are currently two main ways to access GPU-deterministic functionality in TensorFlow for most
            # deep learning applications. 
            # 2.1) The first way is to use an NVIDIA NGC TensorFlow container. - https://www.nvidia.com/en-us/gpu-cloud/containers/
            # 2.2. The second way is to use version 1.14, 1.15, or 2.0 of stock TensorFlow with GPU support, 
            #      plus the application of a patch supplied in this repo.

# # # Ensure Deterministic behaviour
import random
import numpy as np
import tensorflow as tf
os.environ['TF_DETERMINISTIC_OPS'] = '1'

# Now using tensorflow 2.1.0, so no need to patch
# from tfdeterminism import patch
#patch()

seed = 42
os.environ['PYTHONHASHSEED']=str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
############################################

In [ ]:
# utils
from distutils.version import LooseVersion
from tqdm import tqdm_notebook
from datetime import datetime
from time import time

import warnings
import pickle
import gc
import sys
from json import dumps
import itertools
import re

# Data
import pandas as pd
import spacy

# Viz
import matplotlib.pyplot as plt

# Machine Learning
import tensorflow.keras.backend as K
from keras.preprocessing.text import one_hot, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dropout, Bidirectional, LSTM, Flatten, Dense, Reshape, Layer, GlobalAveragePooling1D
from tensorflow.keras.layers import LayerNormalization
from keras.layers.embeddings import Embedding
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.callbacks.callbacks import EarlyStopping
#from keras.callbacks import EarlyStopping, TensorBoard
#from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from keras_self_attention import SeqSelfAttention
from keras_multi_head import MultiHead, MultiHeadAttention

from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# NLP Models
from gensim.models import Word2Vec, KeyedVectors
#w2v_models_path = 'C:/Users/arthu/Desktop/22032020 - Experimentos/05. Organizado/02. Notebooks/models/'
w2v_models_path = 'D:/03. Documentos/Mestrado/Dissertação/07 .Dissertação Final/02. Experimentos/02. Word Embbedings/w2v/'

In [ ]:
# METRICS
# def f1_score(y_true, y_pred):

#     # Count positive samples.
#     c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
#     c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

# #     # If there are no true samples, fix the F1 score at 0.
# #     if c3 == 0:
# #         return 0

#     # How many selected items are relevant?
#     precision = c1 / c2

#     # How many relevant items are selected?
#     recall = c1 / c3

#     # Calculate f1_score
#     f1_score = 2 * (precision * recall) / (precision + recall)
#     return f1_score

# def recall(y_true, y_pred):

#     # Count positive samples.
#     c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

# #     # If there are no true samples, fix the F1 score at 0.
# #     if c3 == 0:
# #         return 0

#     # How many relevant items are selected?
#     recall = c1 / c3
    
#     return recall


# def precision(y_true, y_pred):

#     # Count positive samples.
#     c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))

#     # How many selected items are relevant?
#     precision = c1 / c2

#     return precision

# https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
print('tf version: ' + tf.__version__)

In [ ]:
#Variables
current_exp = 'Atendimento-Unbalanced-Binary'
if 'Binary' in current_exp:
    binary = True
else:
    binary = False
    
base_path = 'D:/03. Documentos/Mestrado/22032020 - Experimentos/05. Organizado/03. Datasets/' + current_exp
save_path = 'output'

sentence = 'req-text'
label = 'Atendimento'

x_train_file = 'X_train.csv'
y_train_file = 'y_train.csv'
x_test_file = 'X_test.csv'
y_test_file = 'y_test.csv'

#Load data
X_train = pd.read_csv(os.path.join(base_path, x_train_file), sep=';', encoding='utf-8')
y_train = pd.read_csv(os.path.join(base_path, y_train_file), sep=';', encoding='utf-8')
X_test = pd.read_csv(os.path.join(base_path, x_test_file), sep=';', encoding='utf-8')
y_test = pd.read_csv(os.path.join(base_path, y_test_file), sep=';', encoding='utf-8')

#Checking on data
print(X_train.columns)
print(X_train.shape)
print(y_train[label].value_counts())
print(y_test[label].value_counts())

In [ ]:
# Keep only text columns
X_train.drop(columns=X_train.columns[2:], inplace=True)
X_test.drop(columns=X_train.columns[2:], inplace=True)

In [ ]:
###################################################
X_train['sentence'] = X_train[sentence]
X_test['sentence'] = X_test[sentence]

y_train['label'] = y_train[label]
y_test['label'] = y_test[label]

##################################################################
# CUT DATAFRAME
# factor = 10000
# df = pd.concat([df[df.label=='1'][0:factor], df[df.label=='0'][0:factor]])
##################################################################

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(X_train.shape[0]))

# Report the classes balance.
print('Classes distribuition: \n')
print(y_train[label].value_counts())

# Display 10 random rows from the data.
X_train.sample(10)

In [ ]:
# Looking lengths
lengths = [X_train.sentence.apply(lambda x: len(x.split(' ')))]
perc =[.25, .50, .75, .80, .85, .90, .91, .92, .93, .94, .95, .96, .97, .98, .99] 
lengths[0].describe(percentiles = perc)

# w2v Model for Embedding Layer

In [ ]:
#w2v_cbow_esic_model=KeyedVectors.load(os.path.join(w2v_models_path,'word2vec_sg_hs_DetalhamentoSolicitacao_all_sentences_128.model'))
#w2v_cbow_nilc_model=KeyedVectors.load_word2vec_format(os.path.join(w2v_models_path,'cbow_s300.txt'))
w2v_cbow_nilc_model=KeyedVectors.load_word2vec_format(os.path.join(w2v_models_path,'skip_s300.txt'))

In [ ]:
pretrained_weights = w2v_cbow_nilc_model.wv.syn0
print(pretrained_weights.shape)
max_num_words = pretrained_weights.shape[0]
embed_size = pretrained_weights.shape[1]

# Data Prep

In [ ]:
texts = X_train['sentence'].append(X_test['sentence'])
t_reading = Tokenizer(num_words=max_num_words)
t_reading.fit_on_texts(texts)
sequences_reading = t_reading.texts_to_sequences(texts)


len_sequences = [len(seq) for seq in sequences_reading]

In [ ]:
# An "interface" to matplotlib.axes.Axes.hist() method
n, bins, patches = plt.hist(x=len_sequences, bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)

plt.grid(axis='y', alpha=0.75)
plt.xlabel('Length')
plt.ylabel('Frequency')
plt.title('Sentence Lenght Distribution')
maxfreq = n.max()
# Set a clean upper y-axis limit.
plt.ylim(ymax=np.ceil(maxfreq / 10) * 10 if maxfreq % 10 else maxfreq + 10)

In [ ]:
trunc = 384
len_sequences_truncated = [trunc if seq>=trunc else seq for seq in len_sequences]

In [ ]:
# An "interface" to matplotlib.axes.Axes.hist() method
n, bins, patches = plt.hist(x=len_sequences_truncated, bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)

plt.grid(axis='y', alpha=0.75)
plt.xlabel('Length')
plt.ylabel('Frequency')
plt.title('Sentence Lenght Distribution')
maxfreq = n.max()
# Set a clean upper y-axis limit.
plt.ylim(ymax=np.ceil(maxfreq / 10) * 10 if maxfreq % 10 else maxfreq + 10)

In [ ]:
max_length=384

# Define tokenizer and fit train data
t = Tokenizer(num_words=max_num_words)
t.fit_on_texts(X_train['sentence'].append(X_test['sentence']))
word_index = t.word_index
vocab_size = len(word_index) + 1
print('Found %s unique tokens.' % len(word_index))
    
def get_seqs(text):    
    sequences = t.texts_to_sequences(text)
    padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')
    return padded_sequences

In [ ]:
# prepare target
def prepare_targets(y_train, y_test):
    le = LabelEncoder()
    le.fit(y_train)
    y_train_enc = le.transform(y_train)
    y_test_enc = le.transform(y_test)
    #if binary:
    #    return y_train_enc, y_test_enc
    #else:
    return pd.get_dummies(y_train_enc), pd.get_dummies(y_test_enc)

In [ ]:
# X and Y
label_train, label_test = prepare_targets(y_train.label.values, y_test.label.values)
num_labels = len(set(label_train))
input_train = get_seqs(X_train.sentence)
input_test = get_seqs(X_test.sentence)

# Modeling

In [ ]:
embedding_matrix = np.zeros((vocab_size, embed_size))
for word, i in t.word_index.items():
    try:
        embedding_vector = w2v_cbow_nilc_model.wv.__getitem__(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    # Words not in vocab -> Frequency less than 5 word
    except KeyError as e:
        print(e)

In [ ]:
# https://www.analyticsvidhya.com/blog/2019/11/comprehensive-guide-attention-mechanism-deep-learning/
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

# class MultiHeadSelfAttention(Layer):
#     def __init__(self, embed_dim, num_heads=8):
#         super(MultiHeadSelfAttention, self).__init__()
#         self.embed_dim = embed_dim
#         self.num_heads = num_heads
#         if embed_dim % num_heads != 0:
#             raise ValueError(
#                 f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
#             )
#         self.projection_dim = embed_dim // num_heads
#         self.query_dense = Dense(embed_dim)
#         self.key_dense = Dense(embed_dim)
#         self.value_dense = Dense(embed_dim)
#         self.combine_heads = Dense(embed_dim)

#     def attention(self, query, key, value):
#         score = tf.matmul(query, key, transpose_b=True)
#         dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
#         scaled_score = score / tf.math.sqrt(dim_key)
#         weights = tf.nn.softmax(scaled_score, axis=-1)
#         output = tf.matmul(weights, value)
#         return output, weights

#     def separate_heads(self, x, batch_size):
#         x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
#         return tf.transpose(x, perm=[0, 2, 1, 3])

#     def call(self, inputs):
#         # x.shape = [batch_size, seq_len, embedding_dim]
#         batch_size = tf.shape(inputs)[0]
#         query = self.query_dense(inputs)  # (batch_size, seq_len, embed_dim)
#         key = self.key_dense(inputs)  # (batch_size, seq_len, embed_dim)
#         value = self.value_dense(inputs)  # (batch_size, seq_len, embed_dim)
#         query = self.separate_heads(
#             query, batch_size
#         )  # (batch_size, num_heads, seq_len, projection_dim)
#         key = self.separate_heads(
#             key, batch_size
#         )  # (batch_size, num_heads, seq_len, projection_dim)
#         value = self.separate_heads(
#             value, batch_size
#         )  # (batch_size, num_heads, seq_len, projection_dim)
#         attention, weights = self.attention(query, key, value)
#         attention = tf.transpose(
#             attention, perm=[0, 2, 1, 3]
#         )  # (batch_size, seq_len, num_heads, projection_dim)
#         concat_attention = tf.reshape(
#             attention, (batch_size, -1, self.embed_dim)
#         )  # (batch_size, seq_len, embed_dim)
#         output = self.combine_heads(
#             concat_attention
#         )  # (batch_size, seq_len, embed_dim)
#         return output
    
# class TokenAndPositionEmbedding(Layer):
#     def __init__(self, maxlen, vocab_size, emded_dim):
#         super(TokenAndPositionEmbedding, self).__init__()
#         self.token_emb = Embedding(input_dim=vocab_size, output_dim=emded_dim)
#         self.pos_emb = Embedding(input_dim=maxlen, output_dim=emded_dim)

#     def call(self, x):
#         maxlen = tf.shape(x)[-1]
#         positions = tf.range(start=0, limit=maxlen, delta=1)
#         positions = self.pos_emb(positions)
#         x = self.token_emb(x)
#         return x + positions
    
# class TransformerBlock(Layer):
#     def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
#         super(TransformerBlock, self).__init__()
#         self.att = MultiHeadSelfAttention(embed_dim, num_heads)
#         self.ffn = Sequential(
#             [Dense(ff_dim, activation="relu"), Dense(embed_dim),]
#         )
#         self.layernorm1 = LayerNormalization(epsilon=1e-6)
#         self.layernorm2 = LayerNormalization(epsilon=1e-6)
#         self.dropout1 = Dropout(rate)
#         self.dropout2 = Dropout(rate)

#     def call(self, inputs):
#         attn_output = self.att(inputs)
#         attn_output = self.dropout1(attn_output, training=False)
#         out1 = self.layernorm1(inputs + attn_output)
#         ffn_output = self.ffn(out1)
#         ffn_output = self.dropout2(ffn_output, training=False)
#         return self.layernorm2(out1 + ffn_output)

In [ ]:
# embed_dim = 32  # Embedding size for each token
# num_heads = 2  # Number of attention heads
# ff_dim = 32  # Hidden layer size in feed forward network inside transformer

# inputs = Input(shape=(max_length,))
# embedding_layer = TokenAndPositionEmbedding(max_length, vocab_size, embed_dim)
# x = embedding_layer(inputs)
# transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
# x = transformer_block(x)
# x = GlobalAveragePooling1D()(x)
# x = Dropout(0.1)(x)
# x = Dense(20, activation="relu")(x)
# x = Dropout(0.1)(x)
# outputs = Dense(2, activation="softmax")(x)

# model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
#https://www.tensorflow.org/api_docs/python/tf/keras/layers/Attention
#https://keras.io/examples/nlp/text_classification_with_transformer/

In [ ]:
# Define model
def train_model(input_train, input_test, label_train, label_test,
                lstm_size=128, dropout=0.2, rec_dropout=0.2, lr=0.005, epochs=50, att_heads=4, max_length=128, 
                vocab_size=None, embed_size=None, emb_trainable=False, batch=128, early_stopping=5,
                save_dir="D:/resultados/checkpoins_solicitacao_keras_mh_att/", best_predefined_f1=0.390):

    # Time now
    now = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Log
    log_dir = save_dir + now
    
    # Model Saver    
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    
    log_file = open(os.path.join(log_dir,"log.txt"), mode="a")
    
    # Save Params
    params = {
        'lstm_size': lstm_size,
        'dropout': dropout,
        'rec_dropout': rec_dropout,
        'lr': lr,
        'epochs': epochs,
        'att_heads': att_heads,
        'max_length': max_length,
        'vocab_size': vocab_size,
        'embed_size': embed_size,
        'emb_trainable': emb_trainable,
        'batch': batch,
        'early_stopping': early_stopping,
        'log_dir': log_dir,
        'best_predefined_f1': best_predefined_f1}
    
    # Saving Parameters
    with open(os.path.join(log_dir, 'params.txt'),'a') as f:
        f.write('\n\n' + ('#'*60))
        f.write('\nParameters:\n')
        f.write('now: ' + str(now))
        f.write('\n' + dumps(params) + '\n')
    
    # input
    inp = Input(shape=(max_length, ))

    # Embedding layer - https://keras.io/layers/embeddings/
    embedding_layer = Embedding(vocab_size,
                                embed_size,                                
                                weights=[embedding_matrix],
                                input_length=max_length,
                                trainable=emb_trainable,
                                name='Embedding')(inp)

    # Bidirectional Layer
    bilstm_layer = Bidirectional(LSTM(
                        units=lstm_size,
                        return_sequences=True,
                        dropout=dropout,
                        recurrent_dropout=rec_dropout,
                        name='LSTM'))(embedding_layer)
    
    bilstm_layer_2 = Bidirectional(LSTM(
                        units=lstm_size,
                        return_sequences=True,
                        dropout=dropout,
                        recurrent_dropout=rec_dropout,
                        name='LSTM'))(bilstm_layer)

    # MultiHead-Attention Layer
    #https://pypi.org/project/keras-multi-head/
    #multiHead_att_layer = MultiHeadAttention(head_num=att_heads, name='Multi-Head-Attention')(bilstm_layer)
    if att_heads:
        #att_layer = MultiHeadSelfAttention(embed_size, att_heads)(bilstm_layer)
        att_layer = MultiHeadAttention(head_num=att_heads, name='Multi-Head-Attention')(bilstm_layer)
    else:
        att_layer = attention()(bilstm_layer)

    dropout_layer = Dropout(0.1)(bilstm_layer_2)

    # # Flatten
    flatten_layer = Flatten(name='Flatten')(dropout_layer)

    #dense_intermed_layer = Dense(128, activation='relu')(flatten_layer)
    #dropout_intermed_2_layer = Dropout(dropout)(dense_intermed_layer)

    # # # # Dense Layer
    #if binary:
    #    dense_layer = Dense(1, activation='sigmoid')(flatten_layer)    
    #else:
    dense_layer = Dense(num_labels, activation='softmax')(flatten_layer)
    
    model = Model(inputs=inp, outputs=dense_layer)
    # model.summary()
    
    # Compile
    model.compile(optimizer=Adam(lr=lr), loss='binary_crossentropy', metrics=['accuracy', precision_m, recall_m, f1_m])
    
    # callbacks
    es_callback = EarlyStopping(monitor='val_loss', patience=early_stopping, verbose=1, mode='min')
    
    # Fitting Model
    model.fit(input_train,
              label_train,
              epochs=epochs,
              batch_size=batch,
              validation_data=(input_test, label_test),
              verbose=0,
              callbacks=[es_callback])
    
    # PLOT LOSS
    plt.title('Loss')
    plt.plot(model.history.history['loss'], label='train')
    plt.plot(model.history.history['val_loss'], label='test')
    plt.legend()
    #plt.show();
    plt.savefig(os.path.join(log_dir,'Loss.png'))
    plt.close()
    
    # Classification
    y_pred = model.predict(input_test, batch_size=batch, verbose=1)
    y_pred_bool = np.argmax(y_pred, axis=1)
    
    #if not binary:
    label_test = np.argmax(label_test.values, axis=1)
    
    # Metrics
    f1 = f1_score(label_test, y_pred_bool, average='weighted')
    print(f"Best Test F1-Score: {f1:.3f}")
    
    print("#"*60 + '\n', file=log_file)
    print(classification_report(label_test, y_pred_bool), file=log_file)
    print("#"*60+ '\n', file=log_file)
    
    # Flush log file
    log_file.flush()
    log_file.close()
    
    # Save final result
    with open(os.path.join(log_dir[:-16], 'output.txt'),'a') as f:
        f.write('\n\n')
        f.write(log_dir)
        f.write('\n')
        f.write(f"Best Test F1-Score: {f1:.3f}")
        
    # save model and architecture to single file
    if f1 > best_predefined_f1:
        model.save(os.path.join(log_dir, "model.h5"))
        print("Saved model to disk")

In [ ]:
# Manual Execution

lstm_size, dropout, rec_dropout, attention_heads, lr = [128, 0.1, 0.1, 2, 5e-05]
batch_size = 64
train_model(input_train, input_test, label_train, label_test, lstm_size, dropout, rec_dropout, lr, epochs=50,
                 att_heads=attention_heads, max_length=max_length, vocab_size=vocab_size, embed_size=embed_size, emb_trainable=False,
                 batch=batch_size, early_stopping=3,
                 save_dir="D:/Outputs_Mestrado/resultados_Atendimento/checkpoins_solicitacao_binary_keras_mh_att/",
                 best_predefined_f1=0.68)

In [ ]:
# Model Params
#lstm_size_list = [128, 256, 512]
lstm_size_list = [512, 1024, 2048]
#attention_heads_list = [2, 4, 8]
#dropout_list = [0.1, 0.25, 0.5]
#rec_dropout_list = [0.1, 0.25, 0.5]
dropout_list = [0.1]
rec_dropout_list = [0.1]
#lr_list = [1e-3, 5e-4, 1e-4, 5e-5, 5e-6]
lr_list = [5e-4, 1e-4, 5e-5, 5e-6]

#all_params = [lstm_size_list] + [dropout_list] + [rec_dropout_list] + [attention_heads_list] + [lr_list]
all_params = [lstm_size_list] + [dropout_list] + [rec_dropout_list] + [lr_list]

for each in itertools.product(*all_params):    
    #lstm_size, dropout, rec_dropout, attention_heads, lr = each
    lstm_size, dropout, rec_dropout, lr = each
        
    
    #if attention_heads==8 or lstm_size==256:
    #    batch_size = 64
    #elif attention_heads> 8 or lstm_size>256:
    #    batch_size = 16
    #else:
    #    batch_size = 256
    batch_size=32
        
    attention_heads = False
    
    # Params
    print('lstm_size: ' + str(lstm_size))
    print('\tdropout: ' + str(dropout))
    print('\trec_dropout: ' + str(rec_dropout))
    #print('\tattention_heads: ' + str(attention_heads))    
    print('\tlr: ' + str(lr))    
    
    # train
    train_model(input_train, input_test, label_train, label_test, lstm_size, dropout, rec_dropout, lr, epochs=50,
                 att_heads=attention_heads, max_length=max_length, vocab_size=vocab_size, embed_size=embed_size, emb_trainable=False,
                 batch=batch_size, early_stopping=3,
                 save_dir="D:/Outputs_Mestrado/resultados_Atendimento/checkpoins_solicitacao_binary_keras_mh_att/",
                 best_predefined_f1=0.68)